In [1]:
import duckdb
import pandas as pd
import numpy as np
import os

PAI_DIR = os.getcwd()  # diretório atual para notebooks jupyter
DATA_RAW = os.path.join(PAI_DIR, "data", "raw")
DATA_PROCESSED = os.path.join(PAI_DIR, "data", "processed")
SCRIPTS = os.path.join(PAI_DIR, "scripts")

In [2]:
!python {os.path.join(SCRIPTS, 'elt_receitas_idaron.py')}

✅ Arquivo salvo: c:\Users\00526862130\Projetos\sefin_geop\notebooks\data\raw\receitas_idaron.parquet


c:\Users\00526862130\Projetos\sefin_geop\notebooks\scripts\gerar_parquet.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=connection)


In [3]:
con = duckdb.connect(database=':memory:')

In [4]:
caminho = os.path.join(DATA_RAW, 'receitas_idaron.parquet')
nome_tabela = 'receitas_idaron'
con.execute(f"""
    CREATE TABLE IF NOT EXISTS {nome_tabela} AS
    SELECT * FROM '{caminho}'
""")

df = con.execute(f"SELECT * FROM {nome_tabela}").df()

df.head(2)

,EXERCICIO,MES,DESCRICAO_GRUPO_SUARE,DESCRICAO_SUBGRUPO,ID_RECEITA,NOME_RECEITA,DESCRICAO_GRUPO,VALOR
0,2007,07,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,5515,D ATIVA NÃO TRIBUTÁRIA IDARON,Dívida Ativa não Tributária,376559.83
1,2007,11,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,5515,D ATIVA NÃO TRIBUTÁRIA IDARON,Dívida Ativa não Tributária,151.10


In [36]:
con.execute(""" SELECT * FROM receitas_idaron LIMIT 5""").df()

,EXERCICIO,MES,DESCRICAO_GRUPO_SUARE,DESCRICAO_SUBGRUPO,ID_RECEITA,NOME_RECEITA,DESCRICAO_GRUPO,VALOR
0,2007,07,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,5515,D ATIVA NÃO TRIBUTÁRIA IDARON,Dívida Ativa não Tributária,376559.83
1,2007,11,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,5515,D ATIVA NÃO TRIBUTÁRIA IDARON,Dívida Ativa não Tributária,151.10
2,2008,01,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,8302,IDARON - FESA CONVÊNIO ICMS 19/22,Taxas,399.13
3,2008,01,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,8312,IDARON - CIS MODELO E - POR TONELADA DE SUBPRO...,Taxas,5598.36
4,2008,01,IDARON,Agência de Defesa Sanitária Agrosilvopastoril ...,8315,IDARON - CERTIFICADO DE TRÂNSITO MAIS DE 50 CO...,Taxas,381.72


In [ ]:
!python {os.path.join(SCRIPTS, 'upf_ro.py')}

In [ ]:
caminho = os.path.join(DATA_RAW, 'upf_ro.parquet')
nome_tabela = 'upf_ro'
con.execute(f"""
    CREATE TABLE IF NOT EXISTS {nome_tabela} AS
    SELECT * FROM '{caminho}'
""")

df = con.execute(f"SELECT * FROM {nome_tabela}").df()

df.head(2)

,ano,valor_upf,var_perc
0,2018,65.21,NaN
1,2019,70.68,8.39


In [ ]:
# Filtros comentados podem ser reativados conforme necessidade:
# WHERE REGEXP_LIKE(F.ID_RECEITA, '^83([0-7][0-9]|8[0-3])$')
# WHERE EXTRACT(MONTH FROM F.DATA_REFERENCIA) BETWEEN 1 AND 5

In [35]:
df = con.execute("""
           
WITH receita_agrupada AS (
    SELECT
        r.EXERCICIO,
        SUM(r.VALOR) AS VALOR,
        u.valor_upf AS UPF_RO,
        u.var_perc AS VAR_PERC_UPF_RO
    FROM receitas_idaron r
    JOIN upf_ro u ON r.EXERCICIO = u.ano
    WHERE r.DESCRICAO_GRUPO IN ('Taxas', 'Serviços')
      AND r.NOME_RECEITA NOT LIKE '%FESA%'
    GROUP BY r.EXERCICIO, u.valor_upf, u.var_perc
),
receita_com_variacao AS (
    SELECT *,
           LAG(VALOR) OVER (ORDER BY EXERCICIO) AS VALOR_ANTERIOR
    FROM receita_agrupada
)
SELECT
    EXERCICIO,
    VALOR,
     ROUND(
        CASE 
            WHEN VALOR_ANTERIOR IS NOT NULL AND VALOR_ANTERIOR != 0 
            THEN ((VALOR - VALOR_ANTERIOR) / VALOR_ANTERIOR) * 100
            ELSE NULL
        END, 2
    ) AS VARIACAO_PERC_RECEITA,
    UPF_RO,
    VAR_PERC_UPF_RO
    FROM receita_com_variacao
ORDER BY EXERCICIO

            """).df()

# df['VAR_PERC_VALOR'] = df['VALOR'].pct_change() * 100

df

,EXERCICIO,VALOR,VARIACAO_PERC_RECEITA,UPF_RO,VAR_PERC_UPF_RO
0,2018,17435606.85,NaN,65.21,NaN
1,2019,24356617.47,39.69,70.68,8.39
2,2020,27929890.90,14.67,74.47,5.36
3,2021,33906685.04,21.40,92.54,24.26
4,2022,36085524.38,6.43,102.48,10.74
5,2023,50309142.28,39.42,108.53,5.90
6,2024,62562198.94,24.36,113.61,4.68
7,2025,43665321.24,-30.20,119.14,4.87
